# Operation Between Tables(Joins, Inserts)

In [1]:
import sqlite3
import random
#connecting to a database with 
conn = sqlite3.connect("HR.db")
c = conn.cursor()

### Note Working with pandas is MUCH Nicer :)

In [19]:
import pandas as pd
df = pd.read_sql("SELECT * FROM employees", conn)
df.head()


,employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,manager_id,department_id
0,100,Steven,King,steven.king@sqltutorial.org,515.123.4567,1987-06-17,4,24000.0,NaN,9
1,101,Neena,Kochhar,neena.kochhar@sqltutorial.org,515.123.4568,1989-09-21,5,17000.0,100.0,9
2,102,Lex,De Haan,lex.de haan@sqltutorial.org,515.123.4569,1993-01-13,5,17000.0,100.0,9
3,103,Alexander,Hunold,alexander.hunold@sqltutorial.org,590.423.4567,1990-01-03,9,9000.0,102.0,6
4,104,Bruce,Ernst,bruce.ernst@sqltutorial.org,590.423.4568,1991-05-21,9,6000.0,103.0,6


In [7]:
query = "SELECT * FROM countries"
print(len(c.execute(query).fetchall()))


25


##  SQL JOINS

An SQL JOIN clause is used to combine rows from two or more tables, based on a common field between them.

#### Getting the tables and table columns from the database

In [11]:
query =("SELECT name FROM sqlite_master WHERE type = 'table'")
print(c.execute(query).fetchall())

[('regions',), ('sqlite_sequence',), ('countries',), ('locations',), ('departments',), ('jobs',), ('employees',), ('dependents',)]


In [15]:
def find_col(table, cursor):
    cursor.execute("SELECT * FROM {}".format(table))
    return [member[0] for member in cursor.description]
    

In [16]:
employee_cols = find_col("employees", c)

print(employee_cols)

['employee_id', 'first_name', 'last_name', 'email', 'phone_number', 'hire_date', 'job_id', 'salary', 'manager_id', 'department_id']


### INNER JOIN

The relational algebra equivalent to a INNER JOIN is intersection. The inner join returns rows if there is, at least, one row in both tables that matches the join condition


In this query we want to combine the employees and the deparments table but we only want the employees in departments 1,2 or 3. We will use a WHERE clause and an IN statement

In [20]:
query = """SELECT first_name, last_name, employees.department_id, department_name 
        FROM employees INNER JOIN departments 
        ON departments.department_id = employees.department_id
        WHERE employees.department_id IN (1,2,3)"""
for row in (c.execute(query).fetchmany(10)):
    print(row)


('Jennifer', 'Whalen', 1, 'Administration')
('Michael', 'Hartstein', 2, 'Marketing')
('Pat', 'Fay', 2, 'Marketing')
('Den', 'Raphaely', 3, 'Purchasing')
('Alexander', 'Khoo', 3, 'Purchasing')
('Shelli', 'Baida', 3, 'Purchasing')
('Sigal', 'Tobias', 3, 'Purchasing')
('Guy', 'Himuro', 3, 'Purchasing')
('Karen', 'Colmenares', 3, 'Purchasing')


### INNER JOIN on three or more tables

Now we query the same output but include the jobs being preformed. This addition 
information is stored in the jobs table and is attainable with the job_id in the employee table

In [22]:
query = """SELECT first_name, last_name, job_title, department_name
        FROM employees e
        INNER JOIN departments d ON d.department_id = e.department_id
        INNER JOIN jobs j ON j.job_id = e.job_id
        WHERE e.department_id IN (1,2,3)"""
for row in (c.execute(query).fetchmany(10)):
    print(row)
#sane output as above with job_title and in different order

('Den', 'Raphaely', 'Purchasing Manager', 'Purchasing')
('Alexander', 'Khoo', 'Purchasing Clerk', 'Purchasing')
('Shelli', 'Baida', 'Purchasing Clerk', 'Purchasing')
('Sigal', 'Tobias', 'Purchasing Clerk', 'Purchasing')
('Guy', 'Himuro', 'Purchasing Clerk', 'Purchasing')
('Karen', 'Colmenares', 'Purchasing Clerk', 'Purchasing')
('Jennifer', 'Whalen', 'Administration Assistant', 'Administration')
('Michael', 'Hartstein', 'Marketing Manager', 'Marketing')
('Pat', 'Fay', 'Marketing Representative', 'Marketing')


### LEFT JOIN

The LEFT JOIN keyword returns all rows from the left table(table(1)) with the matching rows in the 
right table(table(2)). The result is NULL in the right side when there is no match 

The LEFT JOIN clause appears after the FROM clause. The condition that follows
the ON keyword is called the condition

In [25]:
# the left side of this is mandated to be US, UK, CN. There is a one to many 
# relationship. So there are several location in both US and UK but none in 
# CN. The CN entry still is there with NONE values because it is on the left
# side of the join

query = """SELECT c.country_name, c.country_id, l.street_address, l.city
        FROM countries c LEFT JOIN locations l ON l.country_id = c.country_id
        WHERE c.country_id IN('US', 'UK', 'CN')"""
for row in c.execute(query).fetchmany(10):
    print(row)

('China', 'CN', None, None)
('United Kingdom', 'UK', '8204 Arthur St', 'London')
('United Kingdom', 'UK', 'Magdalen Centre, The Oxford Science Park', 'Oxford')
('United States of America', 'US', '2014 Jabberwocky Rd', 'Southlake')
('United States of America', 'US', '2011 Interiors Blvd', 'South San Francisco')
('United States of America', 'US', '2004 Charade Rd', 'Seattle')


### SELF JOIN

Sometime you need to match row from the same table. In this next example we are 
looking to return a list of all employees with there manager name. But we don't have a table with
the managers name connected to the manager_id. The manager's id is just their 
employee id. So what we will do is a self join

In [34]:
query = """SELECT e.first_name || ' ' || e.last_name AS employee,
                  m.first_name || ' ' || m.last_name AS manager
           FROM employees e INNER JOIN employees m 
           ON m.employee_id = e.manager_id
           ORDER BY manager"""
#ok this creates everyone as a manager then checks to see if that "manager" 
#employee is a match to the employee's manager's id. Then it returns the 
#two columns employee and manager
for row in c.execute(query).fetchmany(10):
    print(row)


('Bruce Ernst', 'Alexander Hunold')
('David Austin', 'Alexander Hunold')
('Valli Pataballa', 'Alexander Hunold')
('Diana Lorentz', 'Alexander Hunold')
('Alexander Khoo', 'Den Raphaely')
('Shelli Baida', 'Den Raphaely')
('Sigal Tobias', 'Den Raphaely')
('Guy Himuro', 'Den Raphaely')
('Karen Colmenares', 'Den Raphaely')
('Alexander Hunold', 'Lex De Haan')


This result only returns the employees that have managers if you wanted all employees
whether or not they have a manager then do a LEFT JOIN 

### FULL OUTER JOIN (Not supported with sqlite3)

The full outer join includes all rows from the joined tables where or not the table has the matching row. If the rows in the joined tables do not match, the result set of the full outer join contains NULL values for every column of the table that lacks a matching row

In [24]:
# #creating a new table for simplification
# table1 = """CREATE TABLE employee1 (employee_id INTEGER PRIMARY KEY,
#     employee_name VARCHAR (255) NOT NULL,
#     department_id INTEGER)"""
# table2 ="""CREATE TABLE department1 (department_id INTEGER PRIMARY KEY,
#     department_name VARCHAR (255) NOT NULL
# )"""
# c.execute(table1)
# c.execute(table2)
# conn.commit()


In [25]:
# input1= """INSERT INTO department1 (department_id, department_name) 
#             VALUES(?,?)"""
# dep = [
#     (1, 'loading'),
#     (2, 'shipping'),
#     (3, 'recieving')
#     ]
# input2= """INSERT INTO employee1(employee_id, employee_name, department_id)
#             VALUES(?,?,?)"""
# emp =[
#     (1, 'Peter Pan', 1), 
#     (2, 'Spider Man', 1),
#     (3, 'Super Man', 2), 
#     (4 , "Wonder Women", "NULL")
# ]
# c.executemany(input1, dep)
# c.executemany(input2, emp)
# conn.commit()


In [4]:
query ="SELECT * FROM department1"
print(c.execute(query).fetchall())

[(1, 'loading'), (2, 'shipping'), (3, 'recieving')]


<table>
    <tr>
        <th>department_name<th/>
        <th>employee_name</th>
    </tr>
    <tr>
        <th>loading<th/>
        <th>Peter Pan</th>
    </tr>
    <tr>
        <th>loading<th/>
        <th>Spider Man</th>
    </tr>
    <tr>
        <th>shipping<th/>
        <th>Super Man</th>
    </tr>
    <tr>
        <th>NULL<th/>
        <th>Wonder Women</th>
    </tr>
    <tr>
        <th>receiving<th/>
        <th>NULL</th>
    </tr>
</table>
 

### SQL CROSS JOIN

Cartesian product of two or more tables (Column of table) A $\times$ (Row of table B) and returns a set of all ordered pairs.

In [6]:
query = "SELECT department_name, employee_name FROM department1 CROSS JOIN employee1"
for row in c.execute(query).fetchall():
    print(row)
 

('loading', 'Peter Pan')
('loading', 'Spider Man')
('loading', 'Super Man')
('loading', 'Wonder Women')
('shipping', 'Peter Pan')
('shipping', 'Spider Man')
('shipping', 'Super Man')
('shipping', 'Wonder Women')
('recieving', 'Peter Pan')
('recieving', 'Spider Man')
('recieving', 'Super Man')
('recieving', 'Wonder Women')


### SQL GROUP BY

The GROUP BY clause is an optional clause of the SELECT statement that combines rows into groups based on matching values in specified columns. One row is returned for each group. You often use the GROUP BY in conjunction with an aggregate function such as MIN, MAX, AVG, SUM, or COUNT to calculate a measure that provides the information for each group

In [20]:
query = '''SELECT e.department_id, department_name,
    COUNT(employee_id) headcount
    FROM employees e
    INNER JOIN departments d ON d.department_id = e.department_id
    GROUP BY e.department_id
    ORDER BY headcount DESC
        '''
print(r'(department_id, department_name, rolecount)')
for row in c.execute(query).fetchall():
    print(row)

(department_id, department_name, rolecount)
(5, 'Shipping', 7)
(3, 'Purchasing', 6)
(8, 'Sales', 6)
(10, 'Finance', 6)
(6, 'IT', 5)
(9, 'Executive', 3)
(2, 'Marketing', 2)
(11, 'Accounting', 2)
(1, 'Administration', 1)
(4, 'Human Resources', 1)
(7, 'Public Relations', 1)


### GROUP BY with HAVING

In [23]:
query= """SELECT  e.department_id, department_name,
        COUNT(employee_id) rolecount
        FROM employees e INNER JOIN departments d ON d.department_id = e.department_id
        GROUP BY e.department_id
        HAVING rolecount > 5
        ORDER BY rolecount """
print(r'(department_id, department_name, rolecount)')
for row in c.execute(query).fetchall():
      print(row)


(department_id, department_name, rolecount)
(3, 'Purchasing', 6)
(8, 'Sales', 6)
(10, 'Finance', 6)
(5, 'Shipping', 7)


### SQL UNION operator

To use the UNION operator, you write the dividual SELECT statements and join them by the keyword UNION.  The columns returned by the SELECT statements must have the same or convertible data type, size, and be the same order

The database system processes the query by executing two SELECT statements first. Then, it combines two individual result sets into one and eliminates duplicate rows. To eliminate the duplicate rows, the database system sorts the combined result set by every column and scans it for the matching rows located next to one another. UNION combines rows of the table where JOINs' combine columns.

EXAMPLE This example is not a good way to find a list of parents and children. One it doesn't take 
into account that last name are not always unique amoung your employees, so you could have two employees with the same last name and both or only one has children

In [29]:
query = """SELECT first_name, last_name FROM employees 
           UNION 
           SELECT first_name, last_name FROM dependents
           ORDER BY last_name"""
for row in c.execute(query).fetchmany(10):
    print(row)

('David', 'Austin')
('Fred', 'Austin')
('Hermann', 'Baer')
('Kirsten', 'Baer')
('Sandra', 'Baida')
('Shelli', 'Baida')
('Sarah', 'Bell')
('John', 'Chen')
('Matthew', 'Chen')
('Karen', 'Colmenares')


### SQL INTERSECT

This operater returns rows that both contain the same information. 

In [32]:
query = '''SELECT last_name
           FROM employees  WHERE department_id IN(11)
           INTERSECT
           SELECT last_name
           FROM dependents'''
for row in c.execute(query).fetchall():
    print(row)

('Gietz',)
('Higgins',)


### MINUS (not supported with sqlite3)

This operator allows you to subtract one query result from another.